In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("spam.csv")
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df.Category.value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [4]:
df['spam'] = df['Category'].apply(lambda x: 1 if x=='spam' else 0)
df.headad()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.Message,df.spam,test_size=0.2)

# IMPLEMENTING COUNT VECTORIZER FOR BOW MODEL

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
x_train_cv = v.fit_transform(X_train.values)

In [8]:
x_train_cv.shape

(4457, 7766)

In [9]:
v.get_feature_names_out()[1000:1050]

array(['any', 'anybody', 'anyhow', 'anymore', 'anyone', 'anyplaces',
       'anythiing', 'anythin', 'anything', 'anytime', 'anyway', 'anyways',
       'anywhere', 'aom', 'apartment', 'apeshit', 'aphex', 'apnt', 'apo',
       'apologetic', 'apologise', 'apology', 'app', 'apparently',
       'appeal', 'appear', 'appendix', 'applausestore', 'applebees',
       'apples', 'application', 'apply', 'applyed', 'applying',
       'appointment', 'appreciate', 'appreciated', 'approaches',
       'approaching', 'appropriate', 'approve', 'approved', 'approx',
       'apps', 'appt', 'appy', 'april', 'aproach', 'apt', 'aptitude'],
      dtype=object)

In [10]:
v.vocabulary_

{'as': 1090,
 'usual': 7250,
 'iam': 3586,
 'fine': 2853,
 'happy': 3346,
 'amp': 958,
 'doing': 2392,
 'well': 7478,
 'serious': 6044,
 'what': 7499,
 'like': 4118,
 'proper': 5463,
 'tongued': 6992,
 'her': 3421,
 'can': 1628,
 'you': 7730,
 'plz': 5262,
 'tell': 6777,
 'me': 4413,
 'the': 6837,
 'ans': 989,
 'bslvyl': 1537,
 'sent': 6036,
 'via': 7298,
 'fullonsms': 3042,
 'com': 1899,
 'class': 1823,
 'of': 4892,
 'lt': 4254,
 'gt': 3270,
 'reunion': 5771,
 'valentines': 7264,
 'day': 2175,
 'know': 3976,
 'its': 3781,
 'early': 2502,
 'but': 1577,
 'have': 3366,
 'hundreds': 3563,
 'handsomes': 3327,
 'and': 969,
 'beauties': 1277,
 'to': 6958,
 'wish': 7562,
 'so': 6297,
 'thought': 6888,
 'finish': 2856,
 'off': 4894,
 'aunties': 1146,
 'uncles': 7162,
 '1st': 339,
 'love': 4231,
 'my': 4667,
 'little': 4158,
 'pocy': 5279,
 'bell': 1313,
 'am': 944,
 'sorry': 6346,
 'won': 7600,
 '000': 1,
 'cash': 1670,
 'or': 4979,
 'prize': 5423,
 'claim': 1816,
 'call09050000327': 1606,
 'h

In [11]:
x_train_np = x_train_cv.toarray()
x_train_np

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [22]:
np.where(x_train_np[1]!=0)

(array([3421, 4118, 5463, 6044, 6992, 7499], dtype=int64),)

In [27]:
x_train_np[1][7499]

1

In [29]:
X_train[:4][3910]

'Serious? What like proper tongued her'

In [30]:
v.get_feature_names_out()[4118]

'like'

# Train the naive bayes model

In [31]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(cv,y_train)

MultinomialNB()

In [33]:
x_test_cv=v.transform(X_test)

from sklearn.metrics import classification_report
y_pred = model.predict(x_test_cv)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       979
           1       0.96      0.92      0.94       136

    accuracy                           0.99      1115
   macro avg       0.98      0.96      0.97      1115
weighted avg       0.99      0.99      0.99      1115



# Testing the model

In [34]:
emails = [
    'Hey mohan, can we get together to watch footbal game tomorrow?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!'
]

test = v.transform(emails)
model.predict(test)

array([0, 1], dtype=int64)

# We can also train the model using pipeline

In [35]:
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

In [36]:
clf.fit(X_train, y_train) # here we don't need to provide the count vectorized x_train. pipeline will take care of it.

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [37]:
y_pred = clf.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       979
           1       0.96      0.92      0.94       136

    accuracy                           0.99      1115
   macro avg       0.98      0.96      0.97      1115
weighted avg       0.99      0.99      0.99      1115



# EXERCISE

In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from  sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [39]:
df = pd.read_csv("IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [40]:
df['Category'] = df['sentiment'].apply(lambda x: 1 if x=='positive' else 0)
df.head()

,review,sentiment,Category
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1


In [41]:
X_train,X_test,y_train,y_test = train_test_split(df.review,df.Category,test_size=0.2)

In [42]:
df.shape,X_train.shape

((50000, 3), (40000,))

# Random Forest Model

In [50]:
clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('rf', RandomForestClassifier(n_estimators=50,criterion='entropy'))
])

clf.fit(X_train,y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('rf',
                 RandomForestClassifier(criterion='entropy', n_estimators=50))])

In [51]:
y_pred = clf.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.83      0.85      0.84      4958
           1       0.85      0.83      0.84      5042

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000



# KNN Model

In [52]:
clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('knn', KNeighborsClassifier(n_neighbors=10,metric='euclidean'))
])

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.65      0.66      0.66      4958
           1       0.66      0.66      0.66      5042

    accuracy                           0.66     10000
   macro avg       0.66      0.66      0.66     10000
weighted avg       0.66      0.66      0.66     10000



# MULTINOMIAL NAIVE Model

In [53]:
clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.82      0.88      0.85      4958
           1       0.87      0.81      0.84      5042

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.85      0.84      0.84     10000



# WHY KNN DIDN'T PERFORM WELL??
### 1. As Machine learning algorithms does not work on Text data directly, we need to convert them into numeric vector and feed that into models while training.
### 2. In this process, we convert text into a very high dimensional numeric vector using the technique of Bag of words.
### 3. Model like K-Nearest Neighbours(KNN) doesn't work well with high dimensional data because with large number of dimensions, it becomes difficult for the algorithm to calculate distance in each dimension. In higher dimensional space, the cost to calculate distance becomes expensive and hence impacts the performance of model.
### 4.The easy calculation of probabilities for the words in corpus(Bag of words) and storing them in contigency table is the major reason for the Multinomial NaiveBayes to be a text classification friendly algorithm.
### 5.As Random Forest uses Bootstrapping(Row and column Sampling) with many decision tree and overcomes the high variance and overfitting of high dimensional data and also uses feature importance of words for better classifing the categories.
Machine Learning is like trial and error scientific method, where we keep trying all the possible algorithms we have and select the one which give good results and satisfy the requirements like latency, interpretability etc.